In [2]:
import pygame
import random
import time

# Initialize pygame
pygame.init()

# Constants
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 700
GRID_WIDTH = 10
GRID_HEIGHT = 20
BLOCK_SIZE = 30
GRID_OFFSET_X = (SCREEN_WIDTH - GRID_WIDTH * BLOCK_SIZE) // 2
GRID_OFFSET_Y = SCREEN_HEIGHT - GRID_HEIGHT * BLOCK_SIZE - 50

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (128, 128, 128)
COLORS = [
    (0, 255, 255),  # I (cyan)
    (0, 0, 255),    # J (blue)
    (255, 165, 0),  # L (orange)
    (255, 255, 0),  # O (yellow)
    (0, 255, 0),    # S (green)
    (128, 0, 128),  # T (purple)
    (255, 0, 0)     # Z (red)
]

# Tetromino shapes
SHAPES = [
    [[1, 1, 1, 1]],  # I
    
    [[1, 0, 0],
     [1, 1, 1]],     # J
    
    [[0, 0, 1],
     [1, 1, 1]],     # L
    
    [[1, 1],
     [1, 1]],        # O
    
    [[0, 1, 1],
     [1, 1, 0]],     # S
    
    [[0, 1, 0],
     [1, 1, 1]],     # T
    
    [[1, 1, 0],
     [0, 1, 1]]      # Z
]

# Game setup
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Tetris")
clock = pygame.time.Clock()
font = pygame.font.SysFont('Arial', 30)

class Tetromino:
    def __init__(self, x, y, shape):
        self.x = x
        self.y = y
        self.shape = shape
        self.color = COLORS[SHAPES.index(shape)]
        self.rotation = 0

def create_grid():
    return [[0 for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]

def rotate(shape):
    return [[shape[y][x] for y in range(len(shape)-1, -1, -1)] for x in range(len(shape[0]))]

def valid_space(shape, grid, x, y):
    for i, row in enumerate(shape):
        for j, cell in enumerate(row):
            if cell:
                if (x + j < 0 or x + j >= GRID_WIDTH or 
                    y + i >= GRID_HEIGHT or 
                    (y + i >= 0 and grid[y + i][x + j])):
                    return False
    return True

def draw_grid(surface, grid):
    for y in range(GRID_HEIGHT):
        for x in range(GRID_WIDTH):
            if grid[y][x]:
                pygame.draw.rect(surface, COLORS[grid[y][x] - 1], 
                                (GRID_OFFSET_X + x * BLOCK_SIZE, 
                                 GRID_OFFSET_Y + y * BLOCK_SIZE, 
                                 BLOCK_SIZE, BLOCK_SIZE), 0)
    
    # Draw grid lines
    for y in range(GRID_HEIGHT + 1):
        pygame.draw.line(surface, GRAY, 
                         (GRID_OFFSET_X, GRID_OFFSET_Y + y * BLOCK_SIZE),
                         (GRID_OFFSET_X + GRID_WIDTH * BLOCK_SIZE, GRID_OFFSET_Y + y * BLOCK_SIZE))
    
    for x in range(GRID_WIDTH + 1):
        pygame.draw.line(surface, GRAY, 
                         (GRID_OFFSET_X + x * BLOCK_SIZE, GRID_OFFSET_Y),
                         (GRID_OFFSET_X + x * BLOCK_SIZE, GRID_OFFSET_Y + GRID_HEIGHT * BLOCK_SIZE))

def draw_piece(surface, piece):
    shape = piece.shape
    for i, row in enumerate(shape):
        for j, cell in enumerate(row):
            if cell:
                pygame.draw.rect(surface, piece.color, 
                                (GRID_OFFSET_X + (piece.x + j) * BLOCK_SIZE, 
                                 GRID_OFFSET_Y + (piece.y + i) * BLOCK_SIZE, 
                                 BLOCK_SIZE, BLOCK_SIZE), 0)

def get_new_piece():
    shape = random.choice(SHAPES)
    return Tetromino(GRID_WIDTH // 2 - len(shape[0]) // 2, 0, shape)

def update_grid(grid, piece):
    for i, row in enumerate(piece.shape):
        for j, cell in enumerate(row):
            if cell and piece.y + i >= 0:
                grid[piece.y + i][piece.x + j] = COLORS.index(piece.color) + 1

def clear_lines(grid, score):
    lines_cleared = 0
    y = GRID_HEIGHT - 1
    while y >= 0:
        if all(grid[y]):
            lines_cleared += 1
            grid.pop(y)
            grid.insert(0, [0 for _ in range(GRID_WIDTH)])
        else:
            y -= 1
    
    # Add to score based on lines cleared
    if lines_cleared == 1:
        score += 100
    elif lines_cleared == 2:
        score += 300
    elif lines_cleared == 3:
        score += 500
    elif lines_cleared == 4:
        score += 800
    
    return score

def draw_text(surface, text, size, x, y):
    font = pygame.font.SysFont('Arial', size)
    text_surface = font.render(text, True, WHITE)
    text_rect = text_surface.get_rect(center=(x, y))
    surface.blit(text_surface, text_rect)

def game_over(score):
    screen.fill(BLACK)
    draw_text(screen, "GAME OVER", 50, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 50)
    draw_text(screen, f"Score: {score}", 30, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 20)
    draw_text(screen, "Press any key to quit", 20, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + 70)
    pygame.display.update()
    
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
            if event.type == pygame.KEYDOWN:
                waiting = False
    
    pygame.quit()

def main():
    grid = create_grid()
    current_piece = get_new_piece()
    next_piece = get_new_piece()
    score = 0
    fall_time = 0
    fall_speed = 0.5  # seconds per block
    level_time = 0
    run = True
    
    while run:
        # Game speed
        delta_time = clock.get_rawtime() / 1000
        level_time += delta_time
        fall_time += delta_time
        clock.tick()
        
        # Increase speed every 30 seconds
        if level_time > 30:
            level_time = 0
            fall_speed = max(0.1, fall_speed - 0.05)
        
        # Piece falling
        if fall_time >= fall_speed:
            fall_time = 0
            current_piece.y += 1
            if not valid_space(current_piece.shape, grid, current_piece.x, current_piece.y):
                current_piece.y -= 1
                update_grid(grid, current_piece)
                score = clear_lines(grid, score)
                current_piece = next_piece
                next_piece = get_new_piece()
                if not valid_space(current_piece.shape, grid, current_piece.x, current_piece.y):
                    game_over(score)
                    run = False
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                return
            
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    current_piece.x -= 1
                    if not valid_space(current_piece.shape, grid, current_piece.x, current_piece.y):
                        current_piece.x += 1
                
                if event.key == pygame.K_RIGHT:
                    current_piece.x += 1
                    if not valid_space(current_piece.shape, grid, current_piece.x, current_piece.y):
                        current_piece.x -= 1
                
                if event.key == pygame.K_DOWN:
                    current_piece.y += 1
                    if not valid_space(current_piece.shape, grid, current_piece.x, current_piece.y):
                        current_piece.y -= 1
                
                if event.key == pygame.K_UP:
                    rotated = rotate(current_piece.shape)
                    if valid_space(rotated, grid, current_piece.x, current_piece.y):
                        current_piece.shape = rotated
        
        # Drawing
        screen.fill(BLACK)
        
        # Draw score
        draw_text(screen, f"Score: {score}", 30, SCREEN_WIDTH // 2, 30)
        
        # Draw next piece preview
        draw_text(screen, "Next:", 30, SCREEN_WIDTH // 4 * 3, 150)
        for i, row in enumerate(next_piece.shape):
            for j, cell in enumerate(row):
                if cell:
                    pygame.draw.rect(screen, next_piece.color, 
                                    (SCREEN_WIDTH // 4 * 3 + j * BLOCK_SIZE - len(next_piece.shape[0]) * BLOCK_SIZE // 2, 
                                     200 + i * BLOCK_SIZE, 
                                     BLOCK_SIZE, BLOCK_SIZE), 0)
        
        draw_grid(screen, grid)
        draw_piece(screen, current_piece)
        pygame.display.update()
    
    pygame.quit()

if __name__ == "__main__":
    main()